# Importing all the necesary packages



In [1]:
import re
import string
import pickle
import numpy as np
import pandas as pd
from pandas import read_csv
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
import nltk

# Loading the Dataset

In [2]:
trial_data = pd.read_csv("C:\\Users\\aravi\\Desktop\\Desktop\\SemEval-2020-master\\SemEval-2020-master\\TrialData\\data1.csv", sep=',')
train_data = pd.read_csv("C:\\Users\\aravi\\Desktop\\Desktop\\SemEval-2020-master\\SemEval-2020-master\\TrainData\\data_7000_new.csv", sep=',', names=['image_name', 'Image_URL', 'OCR_extracted_text', 'corrected_text', 'Humour', 'Sarcasm', 'offensive', 'Motivational', 'Overall_Sentiment', 'Basis_of_classification'])

In [3]:
train_data.head()

,image_name,Image_URL,OCR_extracted_text,corrected_text,Humour,Sarcasm,offensive,Motivational,Overall_Sentiment,Basis_of_classification
0,10_year_2r94rv.jpg,https://i.imgflip.com/2r94rv.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive,NaN
1,10_year_10-year-challenge_1547788782.jpeg,https://spiderimg.amarujala.com/assets/images/...,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive,NaN
2,10_year_10yearchallenge-5c75f8b946e0fb0001edc7...,https://www.lifewire.com/thmb/8wNfd94_meE9X2cp...,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive,NaN
3,10_year_10-year-challenge-sweet-dee-edition-40...,https://pics.conservativememes.com/10-year-cha...,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive,NaN
4,10_year_10-year-challenge-with-no-filter-47-hi...,https://pics.me.me/10-year-challenge-with-no-f...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral,NaN


In [4]:
Dit={"not_funny":1,"not_sarcastic":2,"not_offensive":3,"funny":4,"general":5,"slight":6,"very_funny":7,"twisted_meaning":8,"very_offensive":9,"hilarious":10,"very_twisted":11,"hateful_offensive":12,"not_motivational":0,"motivational":0,"very_positive":0 ,"positive":0}

In [5]:
def data1(key):
    a=list(train_data[key].to_csv(index=False).split('\n'))
    #print(train_data[['Humour','Sarcasm', 'offensive']].to_csv())
    #print((a))
    hum=[y.replace('\r', '') for y in a]
    hum.pop()
    return hum

def data2(key):
    a=list(trial_data[key].to_csv(index=False).split('\n'))
    #print(train_data[['Humour','Sarcasm', 'offensive']].to_csv())
    #print((a))
    hum=[y.replace('\r', '') for y in a]
    hum.pop()
    return hum

In [6]:
a=data1('Humour')
b=data1('Sarcasm')
c=data1('offensive')

c:\users\aravi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [7]:
len(train_data)

6601

In [8]:
train_append=[]
for i in range(0,len(train_data)):
    for category, num in Dit.items():    
        if num == max(Dit[a[i]],Dit[b[i]],Dit[c[i]]):
            train_append.append(category)
            #print(category)

In [9]:
#set(train_append)

# Aggregating the data in to three classes

In [10]:
Hum=['funny','very_funny','hilarious']
Sar=['twisted_meaning','very_twisted']
off=['slight','very_offensive','hateful_offensive']
gen=['not_funny','not_sarcastic','not_offensive','general']
for i in range(0,len(train_append)):
    if train_append[i] in Hum:
        train_append[i]='Humour'
    if train_append[i] in Sar:
        train_append[i]='Sarcasm'
    if train_append[i] in off:
        train_append[i]='Offensive'
    if train_append[i] in gen:
        train_append[i]='General'
    #print(train_append[i])

In [11]:
#set(train_append)

In [12]:
train_append=pd.Series(train_append)
#print((train_append))

In [13]:
train_data['cat']=train_append

In [14]:
train_data.head()

,image_name,Image_URL,OCR_extracted_text,corrected_text,Humour,Sarcasm,offensive,Motivational,Overall_Sentiment,Basis_of_classification,cat
0,10_year_2r94rv.jpg,https://i.imgflip.com/2r94rv.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive,NaN,Humour
1,10_year_10-year-challenge_1547788782.jpeg,https://spiderimg.amarujala.com/assets/images/...,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive,NaN,General
2,10_year_10yearchallenge-5c75f8b946e0fb0001edc7...,https://www.lifewire.com/thmb/8wNfd94_meE9X2cp...,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive,NaN,Humour
3,10_year_10-year-challenge-sweet-dee-edition-40...,https://pics.conservativememes.com/10-year-cha...,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive,NaN,Offensive
4,10_year_10-year-challenge-with-no-filter-47-hi...,https://pics.me.me/10-year-challenge-with-no-f...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral,NaN,Sarcasm


In [15]:
d=data2('Humour')
e=data2('Sarcasm')
f=data2('offensive')

c:\users\aravi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  # Remove the CWD from sys.path while we load stuff.


In [16]:
trial_append=[]
for i in range(0,len(trial_data)):
    for category, num in Dit.items():    
        if num == max(Dit[d[i]],Dit[e[i]],Dit[f[i]]):
            trial_append.append(category)
            #print(category)

In [17]:
Hum=['funny','very_funny','hilarious']
Sar=['twisted_meaning','very_twisted']
off=['slight','very_offensive','hateful_offensive']
gen=['not_funny','not_sarcastic','not_offensive','general']
for i in range(0,len(trial_append)):
    if trial_append[i] in Hum:
        trial_append[i]='Humour'
    if trial_append[i] in Sar:
        trial_append[i]='Sarcasm'
    if trial_append[i] in off:
        trial_append[i]='Offensive'
    if trial_append[i] in gen:
        trial_append[i]='General'
    #print(trial_append[i])

In [18]:
trial_data['cat']=trial_append

In [19]:
len(trial_data)

914

In [20]:
train_data = train_data[train_data.cat != 'General']
train_data = train_data[~train_data.cat.isnull()]
trial_data = trial_data[trial_data.cat != 'General']
trial_data = trial_data[~trial_data.cat.isnull()]

In [21]:
def clean_tweets(tweet):
    tweet = re.sub('@(\\w{1,15})\b', '', str(tweet))
    tweet = tweet.replace("via ", "")
    tweet = tweet.replace("RT ", "")
    tweet = tweet.lower()
    return tweet
    
def clean_url(tweet):
    tweet = re.sub('http\\S+', '', tweet, flags=re.MULTILINE)   
    return tweet
    
def remove_stop_words(tweet):
    stops = set(stopwords.words("english"))
    stops.update(['.',',','"',"'",'?',':',';','(',')','[',']','{','}'])
    toks = [tok for tok in tweet if not tok in stops and len(tok) >= 3]
    return toks
    
def stemming_tweets(tweet):
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in tweet]
    return stemmed_words

def remove_number(tweet):
    newTweet = re.sub('\\d+', '', tweet)
    return newTweet

def remove_hashtags(tweet):
    result = ''

    for word in tweet.split():
        if word.startswith('#') or word.startswith('@'):
            result += word[1:]
            result += ' '
        else:
            result += word
            result += ' '

    return result

In [22]:
def preprocessing(tweet, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True):

    if ctweets:
        tweet = clean_tweets(tweet)

    if url:
        tweet = clean_url(tweet)

    if hashtag:
        tweet = remove_hashtags(tweet)
    
    twtk = TweetTokenizer(strip_handles=True, reduce_len=True)

    if number:
        tweet = remove_number(tweet)
    
    tokens = [w.lower() for w in twtk.tokenize(tweet) if w != "" and w is not None]

    if swords:
        tokens = remove_stop_words(tokens)

    if stemming:
        tokens = stemming_tweets(tokens)

    text = " ".join(tokens)

    return text

In [23]:
train_text  = train_data['corrected_text'].map(lambda x: preprocessing(x, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True))
s_train     = train_data['cat']

trial_text  = trial_data['corrected_text'].map(lambda x: preprocessing(x, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True))
s_trial     = trial_data['cat']

In [24]:
def bag_of_words(train, test):
    vec = CountVectorizer(analyzer='word', binary=True, min_df=1, max_features=25000)
    train = vec.fit_transform(train).toarray()
    test = vec.transform(test).toarray()
    return train, test

In [25]:
x_train, x_test = bag_of_words(train_text, trial_text)

In [26]:
clf = LinearSVC(C=0.9)

clf.fit(x_train, s_train)

y_pred = clf.predict(x_test)

In [27]:
#print(y_pred)

In [28]:
print("F1.........: %f" %(f1_score(s_trial, y_pred, average="macro")))
print("Precision..: %f" %(precision_score(s_trial, y_pred, average="macro")))
print("Recall.....: %f" %(recall_score(s_trial, y_pred, average="macro")))
print("Accuracy...: %f" %(accuracy_score(s_trial, y_pred)))

F1.........: 0.464096
Precision..: 0.686749
Recall.....: 0.473062
Accuracy...: 0.550409


In [29]:
from sklearn.svm import SVC
clf1 = SVC(kernel = "linear", probability = True)
clf1.fit(x_train, s_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [30]:
y_pred = clf1.predict(x_test)

In [31]:
print("F1.........: %f" %(f1_score(s_trial, y_pred, average="macro")))
print("Precision..: %f" %(precision_score(s_trial, y_pred, average="macro")))
print("Recall.....: %f" %(recall_score(s_trial, y_pred, average="macro")))
print("Accuracy...: %f" %(accuracy_score(s_trial, y_pred)))

F1.........: 0.447237
Precision..: 0.701741
Recall.....: 0.461111
Accuracy...: 0.543597


In [32]:
y_prob=clf1.predict_proba(x_test)

In [33]:
y_prob[:500]

array([[0.34350527, 0.44939564, 0.20709909],
       [0.33756903, 0.46585389, 0.19657708],
       [0.33346125, 0.48321572, 0.18332304],
       ...,
       [0.34350527, 0.44939564, 0.20709909],
       [0.34350527, 0.44939564, 0.20709909],
       [0.34189214, 0.47003245, 0.18807542]])

In [34]:
def quantitation(arr):
    
    if arr<0.25:
        a = 1
    elif arr>0.25 and arr<0.50:
        a=(2)
    elif arr>0.50 and arr<0.75:
        a=(3)
    elif arr>0.75 and arr<1:
        a=(4)
    
    
    return a

In [35]:
quantitation(max(y_prob[0]))

2

In [36]:
max(y_prob[0])

0.44939564309347124